## Ajusta parâmetros para SVM, RF e XGBoost

In [ ]:
!pip install pyefd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/shape_based_CRIC_Hclassifier")

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
import os
import sys
import csv
from collections import Counter
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from skimage import morphology, measure
from skimage.draw import polygon, polygon_perimeter
from scipy.spatial.distance import cdist
from scipy.stats import kurtosis

import pyefd
from pyefd import elliptic_fourier_descriptors, normalize_efd

from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import feature_selection as fs
from sklearn import preprocessing

from datetime import datetime

# pay attention to capitalization below!
from spFSR import SpFSR
from imblearn.over_sampling import SMOTE, ADASYN, SVMSMOTE, BorderlineSMOTE

from itertools import cycle
from random import randint
from random import sample

import xgboost as xgb

import functions, shapeFeatures

In [ ]:
Bethesda_classes = {'Normal':0, 'ASC-US':1, 'ASC-H':2, 'LSIL':3,'HSIL':4, 'Invasive Carcinoma':5}
Bethesda_idx_classes = {0: 'Normal', 1:'ASC-US', 2:'ASC-H', 3:'LSIL',4: 'HSIL', 5:'Invasive Carcinoma'}

### Features:

In [ ]:
len(functions.list_all_features(20)), len(functions.list_all_nucleus_features(20)), len(functions.list_all_cyto_features(20)), len(functions.list_all_EFD_features(20))


(200, 98, 98, 154)

### Lê arquivo (features):

In [ ]:
N_EFD_COEFFS = 20

In [ ]:
#df = pd.read_csv('dataCRIC.csv', sep='|', header=0)
df = pd.read_csv('/content/drive/MyDrive/shape_based_CRIC_Hclassifier/dataCRIC.csv', sep='|', header=0)
df = shapeFeatures.normalize_dataset(df, n_efd_coeffs= N_EFD_COEFFS)


In [ ]:
df

,image_id,cell_id,areaN,eccenN,extentN,periN,maxAxN,minAxN,compacN,circuN,...,efdC73,efdC74,efdC75,efdC76,efdC77,ratio_NC,ratio_NC_per,ratio_NC_hArea,nucleus_position,bethesda
0,1.0,14796.0,0.007403,0.606414,0.842276,0.031326,0.036711,0.083827,0.024126,0.863522,...,0.156532,0.321490,0.859931,0.382225,0.302311,0.017236,0.115646,0.016730,0.099098,0.0
1,1.0,14797.0,0.009371,0.541484,0.808102,0.035945,0.039493,0.095649,0.018763,0.891071,...,0.150931,0.317850,0.859539,0.369221,0.299938,0.010656,0.089517,0.010168,0.121456,0.0
2,1.0,14798.0,0.007275,0.313794,0.884418,0.029457,0.025365,0.097062,0.018895,0.890378,...,0.153458,0.315454,0.861978,0.367980,0.302076,0.007400,0.073961,0.006897,0.043773,0.0
3,1.0,14799.0,0.010570,0.489270,0.778201,0.040199,0.040900,0.103428,0.021001,0.879403,...,0.157254,0.319562,0.860312,0.382943,0.303682,0.028309,0.154784,0.027752,0.128927,0.0
4,1.0,14801.0,0.009115,0.296366,0.842276,0.036826,0.031283,0.106376,0.024456,0.861871,...,0.147193,0.314554,0.860292,0.359672,0.300094,0.021569,0.129992,0.020738,0.069245,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,399.0,11539.0,0.028927,0.838629,0.579720,0.094256,0.134656,0.118369,0.041270,0.784196,...,0.152113,0.318217,0.859606,0.370535,0.299400,0.206211,0.149094,0.085315,1.943974,5.0
3229,399.0,11540.0,0.025333,0.874227,0.645532,0.094611,0.135676,0.100778,0.064959,0.692461,...,0.150010,0.322108,0.859366,0.375599,0.300755,0.245537,0.247086,0.136756,0.916374,5.0
3230,400.0,11535.0,0.017031,0.910741,0.390863,0.092280,0.136377,0.078990,0.131860,0.507358,...,0.153944,0.319972,0.860379,0.372458,0.303245,0.191508,0.471092,0.188343,0.326252,5.0
3231,400.0,11536.0,0.020797,0.949322,0.528816,0.105709,0.169029,0.065965,0.137720,0.494794,...,0.151898,0.320620,0.859608,0.370771,0.299599,0.111823,0.078186,0.023058,3.177247,5.0


In [ ]:
# Separa dados por classe de maneira balanceada:
data_normal = df[df['bethesda'] == 0].copy()
data_normal.set_index((i for i in range(data_normal.shape[0])), inplace=True)

data_ascus = df[df['bethesda'] == 1].copy()
data_ascus.set_index((i for i in range(data_ascus.shape[0])), inplace=True)

data_asch = df[df['bethesda'] == 2].copy()
data_asch.set_index((i for i in range(data_asch.shape[0])), inplace=True)

data_lsil = df[df['bethesda'] == 3].copy()
data_lsil.set_index((i for i in range(data_lsil.shape[0])), inplace=True)

data_hsil = df[df['bethesda'] == 4].copy()
data_hsil.set_index((i for i in range(data_hsil.shape[0])), inplace=True)

data_car = df[df['bethesda'] == 5].copy()
data_car.set_index((i for i in range(data_car.shape[0])), inplace=True)

print("--- Totais por classe --- ")
print("Normal...: ", data_normal.values.shape[0])
print("ASC-Us...: ", data_ascus.values.shape[0])
print("ASC-H....: ", data_asch.values.shape[0])
print("LSIL.....: ", data_lsil.values.shape[0])
print("HSIL.....: ", data_hsil.values.shape[0])
print("Carcinoma: ", data_car.values.shape[0])


--- Totais por classe --- 
Normal...:  862
ASC-Us...:  286
ASC-H....:  536
LSIL.....:  598
HSIL.....:  874
Carcinoma:  77


#### Gera dataframes: dados (data), classes (target) e Ids (image/cell)

In [ ]:
# Monta base (data, target, image/cells ids)
data, target, image_cells_ids= functions.get_database_data_targe_ids(data_normal, data_ascus,
                       data_lsil, data_asch, data_hsil,data_car,
                       functions.list_all_features(N_EFD_COEFFS))


#### Tuning de parâmetros (Classificador)
Feito apenas uma vez com um subconjunto dos dados e para cada um dos métodos (SVM, RF e XGBOOST).

Conjunto de dados para tuning:

   X: merge de todas as features selecionadas para o classificador hierarquico (classificadores 1,2,3, e 4)
  
   y: rótulos das 6 classes Bethesda.

In [ ]:
# Separa dados usando 80% para tuning:
(X_train, X_test, y_train, y_test, image_cells_ids_train, image_cells_ids_test) = train_test_split(data, target, image_cells_ids, test_size=0.2, random_state=45)

Counter(y_test['bethesda'].values),  Counter(y_train['bethesda'].values)

(Counter({1.0: 44, 2.0: 107, 0.0: 172, 3.0: 127, 5.0: 13, 4.0: 184}),
 Counter({4.0: 690, 2.0: 429, 0.0: 690, 1.0: 242, 3.0: 471, 5.0: 64}))

In [ ]:
#### Aplica seleção de features (executada em separado): classificadores (1, 2, 3, 4)
best_features_spfsr_1 = ['ratio_NC','ratio_NC_per','eN','ratio_NC_hArea','ardN','efdC77','efdC1',
                         'efdN58','efdN21','efdN57','efdC55','efdN30','efdC20','efdN50','solidC',
                         'efdC37','efdN29']

best_features_MI_1 =  ['ratio_NC_per','ratio_NC','ratio_NC_hArea','mrdN','eN','maxAxN',
                       'periN', 'ardN','hAreaN','equidiaN','areaN','fdN','compacC','circuC','riN',
                       'areaC','hAreaC','periC','minAxC','fdC','maxAxC','ardC','equidiaC','mrdC',
                       'circuN','compacN','eC','riC','minAxN','convexN']

best_features_spfsr_2 = ['areaC','fdC','eC','ratio_NC','ratio_NC_hArea','equidiaC','maxAxC','periC','compacC','mrdC']


best_features_MI_2 =  ['areaC','equidiaC','fdC','ardC','hAreaC','periC','ratio_NC_hArea',
                       'ratio_NC','mrdC','maxAxC','eC','ratio_NC_per','riC','minAxC','circuC',
                       'compacC','equidiaN','areaN','fdN','hAreaN','periN','minAxN','riN',
                       'nucleus_position','ardN','solidC','maxAxN','mrdN','extentC','eN']


best_features_spfsr_3 =  ['solidN','nucleus_position','efdN40','efdC17','efdN58','compacN','circuN',
                          'efdN43','efdC51','efdN26','efdN76','efdN47','efdN5','efdC24','sdnrlN',
                          'efdN61','efdC65','efdC61','raC','ratio_NC_per','efdN44','sdnrlC',
                          'efdC19','efdN11','efdC8','efdN56','efdC52','efdN55','efdC4','solidC']

best_features_MI_3 = ['efdC39','efdC31','efdN76','efdC33','efdC63','efdN12','efdC41','efdC44',
                      'efdN39','efdN15','efdC56','efdC66','efdN13','efdC14','mrdN','efdN25',
                      'fdC','circuC','efdC51','efdC3','extentN','efdN2','compacC','efdC18',
                      'efdN48','efdN62','minAxN','efdN65','efdC4','efdC24']

best_features_spfsr_4 =  ['ratio_NC_per','ratio_NC','ratio_NC_hArea','areaC','efdC19','efdN45',
                          'efdN73','efdC22','efdC21','efdC30','efdC47','efdN11','efdN50','hAreaN',
                          'efdN2','efdC46']

best_features_MI_4 =  ['ratio_NC','ratio_NC_hArea','ratio_NC_per','areaC','compacC','circuC',
                       'mrdC','solidC','eC','periC','ardC','extentC','nucleus_position','maxAxC',
                       'eccenC','riC','raC','convexN','hAreaC','convexC','efdC32','efdC62',
                       'efdC2','raN','efdN9','efdC52','efdC6','efdN60','efdC22','efdN20']

In [ ]:
len(best_features_MI_4), len(best_features_spfsr_4), type(best_features_spfsr_1)

(30, 16, list)

In [ ]:
#features_sel = list(set(best_features_MI_1 + best_features_MI_2 +
#                        best_features_MI_3 + best_features_MI_4 ))
features_sel = list(set(best_features_spfsr_1 + best_features_spfsr_2 +
                        best_features_spfsr_3 + best_features_spfsr_4))
#len(features_sel), features_sel

##### Funções TUNING:

In [ ]:
#Funções tuning/gridsearch (SVM, RF, XGBoost)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

def grid_search_SVM(model, params, X, y):
    grid_search = GridSearchCV(
        model, params, scoring= ['accuracy', 'f1_weighted'], refit='f1_weighted'
    )
    grid_search.fit(X, y)
    y_pred = grid_search.predict(X)
    print(classification_report(y, y_pred))
    return (grid_search.best_params_)

def grid_search_RF(model, params, X, y):
    grid_search = GridSearchCV(model, param_grid = params,
        scoring= 'accuracy')
    grid_search.fit(X, y)
    y_pred = grid_search.predict(X)
    print(classification_report(y, y_pred))
    return (grid_search.best_params_)

def grid_search_XGB(model, params, X, y):
    folds = 5
    param_comb = 150
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

    random_search = RandomizedSearchCV(model, param_distributions= params,
                        n_iter=param_comb, scoring='accuracy', n_jobs=-1,
                        cv=skf.split(X,y), random_state=1001)

    start_time = functions.timer(None) # Tempo inicial
    random_search.fit(X, y)
    functions.timer(start_time)

    print('\n Best estimator:')
    print(random_search.best_estimator_)
    print('\n Best score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
    print(random_search.best_score_ )
    results = pd.DataFrame(random_search.cv_results_)

    return random_search.best_params_, results


### Funções melhoradas para XGBoost: (função similar à: (https://www.analyticsvidhya.com Assunto:Complete Guide to Parameter Tuning in XGBoost))
def modelfit(alg, x_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(x_train.values, label=y_train.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='merror', early_stopping_rounds=early_stopping_rounds, verbose_eval= None)
        print('Estimadores: ', cvresult.shape[0])
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(x_train, y_train,eval_metric='merror')

    #Predict training set:
    dtrain_predictions = alg.predict(x_train)

    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))


###### Tuning para XGBoost

In [ ]:
## Etapa 1: definindo n_estimadors para learning_rate de 0.1
start_t = functions.timer()
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 num_class= 6,
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X_train, y_train['bethesda'])

functions.timer(start_time=start_t)

/usr/local/lib/python3.8/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


Passou aqui
Estimadores:  148

Model Report
Accuracy : 1

 Time taken: 0 hours 2 minutes and 57.69 seconds.


In [ ]:
## Etapa 2: definindo max_depth e min_child_weight com n_estimadors=148 e learning_rate de 0.1
start_t = functions.timer()
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
alg = xgb.XGBClassifier(learning_rate =0.1, n_estimators=148, max_depth=5,
                                          min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                          objective= 'multi:softmax', num_class= 6, nthread=4, scale_pos_weight=1, seed=27)
gsearch1 = GridSearchCV(estimator = alg, param_grid = param_test1, scoring='accuracy', n_jobs=4, cv=5)

gsearch1.fit(X_train, y_train['bethesda'])
functions.timer(start_time=start_t)


 Time taken: 0 hours 21 minutes and 21.79 seconds.


In [ ]:
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 9, 'min_child_weight': 1}, 0.7173229875357535)

In [ ]:
## Etapa 3/: refinando max_depth com min_child_weight=1 com n_estimadors=148 e learning_rate de 0.1
start_t = functions.timer()
param_test2 = {
 'max_depth':[8,9,10],
 'min_child_weight':[1,2,3]
}

alg = xgb.XGBClassifier(learning_rate =0.2, n_estimators=148, max_depth=9,
                                          min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                          objective= 'multi:softmax', num_class= 6, scale_pos_weight=1, seed=27)
gsearch2 = GridSearchCV(estimator = alg, param_grid = param_test2, scoring='accuracy', n_jobs=-1, cv=5)

gsearch2.fit(X_train, y_train['bethesda'])
functions.timer(start_time=start_t)


 Time taken: 0 hours 15 minutes and 38.6 seconds.


In [ ]:
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 9, 'min_child_weight': 5}, 0.7211914594893318)

In [ ]:
## Etapa 4: definindo gamma max_depth=9 com min_child_weight=1 com n_estimadors=148 e learning_rate de 0.1
start_t = functions.timer()
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
alg = xgb.XGBClassifier(learning_rate =0.1, n_estimators=148, max_depth=9,
                                          min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                          objective= 'multi:softmax', num_class= 6, scale_pos_weight=1, seed=27)

gsearch3 = GridSearchCV(estimator = alg, param_grid = param_test3,  scoring='accuracy', n_jobs=-1, cv=5)

gsearch3.fit(X_train, y_train['bethesda'])
functions.timer(start_time=start_t)


 Time taken: 0 hours 12 minutes and 46.99 seconds.


In [ ]:
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.0}, 0.7173229875357535)

In [ ]:
## Etapa 5: definindo subsample, colsample_bytree
start_t = functions.timer()
param_test4 = {
 'subsample':[i/10.0 for i in range(7,10)],
 'colsample_bytree':[i/10.0 for i in range(7,10)]
}

alg = xgb.XGBClassifier(learning_rate =0.1, n_estimators=148, max_depth=9,
                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                        objective= 'multi:softmax', num_class= 6, scale_pos_weight=1, seed=27)

gsearch4 = GridSearchCV(estimator = alg, param_grid = param_test4,  scoring='accuracy', n_jobs=-1, cv=5)

gsearch4.fit(X_train, y_train['bethesda'])
functions.timer(start_time=start_t)


 Time taken: 0 hours 20 minutes and 48.24 seconds.


In [ ]:
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.7, 'subsample': 0.9}, 0.719259463940315)

In [ ]:
## Etapa 6: definindo reg_alpha  #L1 regularization term on weights#

start_t = functions.timer()
param_test6 = {
 'max_delta_step': [0, 5, 10],
 'reg_alpha':[1e-5, 1e-3,1e-2, 0.1, 0.5]
}

alg = xgb.XGBClassifier(learning_rate =0.1, n_estimators=148, max_depth=9,
                        min_child_weight=1, gamma=0, subsample=0.9, colsample_bytree=0.7,
                        objective= 'multi:softmax', num_class= 6, scale_pos_weight=1, seed=27)

gsearch6 = GridSearchCV(estimator = alg, param_grid = param_test6,  scoring='accuracy', n_jobs=-1, cv=5)

gsearch6.fit(X_train, y_train['bethesda'])
functions.timer(start_time=start_t)


 Time taken: 0 hours 37 minutes and 45.57 seconds.


In [ ]:
gsearch6.best_params_, gsearch6.best_score_

({'max_delta_step': 0, 'reg_alpha': 0.001}, 0.7204162714801012)

In [ ]:
start_t = functions.timer()
#xgb_param = {'learning_rate': 0.1, 'n_estimators': 148, 'max_depth': 9, 'min_child_weight':1,
#              'gamma':0, 'subsample':0.9, 'colsample_bytree':0.7, 'reg_alpha': 0.001}
#alg = xgb.XGBClassifier(objective= 'multi:softmax', num_class= 6, seed=27).set_params(**xgb_param)

xgb7 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=5000,
 max_depth=9,
 min_child_weight=1,
 gamma=0,
 subsample=0.9,
 colsample_bytree=0.7,
 objective= 'multi:softmax',
 num_class= 6,
 nthread=4,
 reg_alpha= 0.001,
 seed=27)
modelfit(xgb7, X_train, y_train['bethesda'])

functions.timer(start_time=start_t)

/usr/local/lib/python3.8/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


Passou aqui
Estimadores:  86

Model Report
Accuracy : 1

 Time taken: 0 hours 2 minutes and 30.3 seconds.


In [ ]:
svm_params = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
    {"kernel": ["poly"], "degree":[2,3,4], "gamma": [1e-3, 1e-4], "coef0":[2e-1, 2e-2] , "C": [1, 10, 100, 1000]},
]
#svm_param =  grid_search_SVM(SVC(), svm_params, X_train[features_sel], y_train['bethesda'])
#print('Best svm params: ', svm_param)

              precision    recall  f1-score   support

         0.0       0.95      0.94      0.94       690
         1.0       0.71      0.21      0.32       242
         2.0       0.61      0.46      0.52       429
         3.0       0.68      0.91      0.78       471
         4.0       0.70      0.85      0.77       690
         5.0       0.89      0.52      0.65        64

    accuracy                           0.75      2586
   macro avg       0.76      0.65      0.66      2586
weighted avg       0.75      0.75      0.73      2586

Best svm params:  {'C': 100, 'kernel': 'linear'}


In [ ]:
rf_params = {'max_depth': [5, 7, 8, 9],
            'min_samples_split': [5, 9, 10, 12],
            'n_estimators': [30, 40, 50, 60, 100]
            }
#rf_param = grid_search_RF(RandomForestClassifier(random_state=0),
#                             rf_params, X_train[features_sel], y_train['bethesda'])
#print('Best Random Forest params: ', rf_param)

              precision    recall  f1-score   support

         0.0       0.98      0.93      0.95       690
         1.0       0.88      0.21      0.35       242
         2.0       0.78      0.70      0.74       429
         3.0       0.69      0.98      0.81       471
         4.0       0.81      0.93      0.87       690
         5.0       1.00      0.42      0.59        64

    accuracy                           0.82      2586
   macro avg       0.86      0.70      0.72      2586
weighted avg       0.84      0.82      0.80      2586

Best Random Forest params:  {'max_depth': 7, 'min_samples_split': 10, 'n_estimators': 50}


In [ ]:
xgb_params = {
        'min_child_weight': [1, 5, 7, 10],
        'gamma': [0, 0.3, 0.5, 1],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6, 7]
        }


#xgb_param, result = grid_search_XGB(xgb.XGBClassifier(learning_rate=0.2,
#                                               n_estimators=200, objective='multi:softprob'),
#                                    xgb_params, X_train[features_sel], y_train['bethesda'])
#print('Best XGBoost params: ', xgb_param)

In [ ]:
## Duração: despresível
svm_param =  {'C': 100, 'kernel': 'linear'}
rf_param = {'max_depth': 7, 'min_samples_split': 10, 'n_estimators': 50}

## Duração ~ 1,50h
xgb_param = {'learning_rate': 0.1, 'n_estimators': 86, 'max_depth': 9, 'min_child_weight':1,
              'gamma':0, 'subsample':0.9, 'colsample_bytree':0.7, 'reg_alpha': 0.001}
